In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os, re
import pandas as pd

BASE = "/kaggle/input/3w-dataset/2.0.0"

paths = []
for root, _, files in os.walk(BASE):
    for f in files:
        if f.endswith(".parquet"):
            paths.append(os.path.join(root, f))

df_files = pd.DataFrame({"path": paths})
df_files["label"] = df_files["path"].str.extract(r"/2\.0\.0/(\d+)/").astype(int)

# parse file name like WELL-00023_20181030045054.parquet
df_files["file"] = df_files["path"].str.split("/").str[-1]
df_files["source"] = df_files["file"].str.extract(r"^(WELL|SIMULATED|DRAWN)")
df_files["well_id"] = df_files["file"].str.extract(r"(WELL-\d+)")
df_files["run_ts"] = df_files["file"].str.extract(r"_(\d{14})")  # may be NaN for DRAWN
df_files["run_ts"] = pd.to_datetime(df_files["run_ts"], format="%Y%m%d%H%M%S", errors="coerce")

df_files.sort_values(["label","source","well_id","run_ts"], inplace=True)
df_files.head(10)


,path,label,file,source,well_id,run_ts
678,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201010207.parquet,WELL,WELL-00001,2017-02-01 01:02:07
863,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201060114.parquet,WELL,WELL-00001,2017-02-01 06:01:14
650,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201110124.parquet,WELL,WELL-00001,2017-02-01 11:01:24
878,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201160311.parquet,WELL,WELL-00001,2017-02-01 16:03:11
813,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201210228.parquet,WELL,WELL-00001,2017-02-01 21:02:28
1136,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170202020343.parquet,WELL,WELL-00001,2017-02-02 02:03:43
949,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170202070239.parquet,WELL,WELL-00001,2017-02-02 07:02:39
801,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170218000146.parquet,WELL,WELL-00001,2017-02-18 00:01:46
1058,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170218050218.parquet,WELL,WELL-00001,2017-02-18 05:02:18
887,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170218100218.parquet,WELL,WELL-00001,2017-02-18 10:02:18


In [3]:
# pick one file (for example first row)
one_path = df_files.iloc[0]["path"]
one_path

'/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20170201010207.parquet'

In [4]:
# read that parquet file
df_one = pd.read_parquet(one_path)

print(df_one.shape)
df_one.head()

(21474, 29)


,ABER-CKGL,ABER-CKP,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,ESTADO-W1,ESTADO-W2,...,PT-P,P-TPT,QBS,QGL,T-JUS-CKP,T-MON-CKP,T-PDG,T-TPT,class,state
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-02-01 01:02:07,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,10074540.0,NaN,0.0,84.64463,NaN,0.0,119.0781,<NA>,<NA>
2017-02-01 01:02:08,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,10074540.0,NaN,0.0,84.63828,NaN,0.0,119.0781,<NA>,<NA>
2017-02-01 01:02:09,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,10074540.0,NaN,0.0,84.63194,NaN,0.0,119.0781,<NA>,<NA>
2017-02-01 01:02:10,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,10074540.0,NaN,0.0,84.62558,NaN,0.0,119.0781,<NA>,<NA>
2017-02-01 01:02:11,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,NaN,10074540.0,NaN,0.0,84.61923,NaN,0.0,119.0781,<NA>,<NA>


In [5]:
df_one.columns


Index(['ABER-CKGL', 'ABER-CKP', 'ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2',
       'ESTADO-PXO', 'ESTADO-SDV-GL', 'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2',
       'ESTADO-XO', 'P-ANULAR', 'P-JUS-BS', 'P-JUS-CKGL', 'P-JUS-CKP',
       'P-MON-CKGL', 'P-MON-CKP', 'P-MON-SDV-P', 'P-PDG', 'PT-P', 'P-TPT',
       'QBS', 'QGL', 'T-JUS-CKP', 'T-MON-CKP', 'T-PDG', 'T-TPT', 'class',
       'state'],
      dtype='object')

In [6]:
import numpy as np
import pandas as pd

def summarize_timeseries(df):
    # keep only numeric sensor columns
    num = df.select_dtypes(include=[np.number])

    feats = {}
    for col in num.columns:
        feats[f"{col}_mean"] = num[col].mean()
        feats[f"{col}_std"]  = num[col].std()
        feats[f"{col}_min"]  = num[col].min()
        feats[f"{col}_max"]  = num[col].max()
        feats[f"{col}_last"] = num[col].iloc[-1]
    return feats

# take a small sample so it's not overwhelming / slow
sample = df_files.sample(200, random_state=42).reset_index(drop=True)

rows = []
for i, r in sample.iterrows():
    df_ts = pd.read_parquet(r["path"])
    feats = summarize_timeseries(df_ts)

    feats["label"] = r["label"]
    feats["source"] = r["source"]
    feats["well_id"] = r["well_id"]
    rows.append(feats)

df_ml = pd.DataFrame(rows)
df_ml.head()


,ABER-CKGL_mean,ABER-CKGL_std,ABER-CKGL_min,ABER-CKGL_max,ABER-CKGL_last,ABER-CKP_mean,ABER-CKP_std,ABER-CKP_min,ABER-CKP_max,ABER-CKP_last,...,class_max,class_last,state_mean,state_std,state_min,state_max,state_last,label,source,well_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0,0,WELL,WELL-00001
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0,0,WELL,WELL-00006
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106,6,0.0,0.0,0,0,0,6,SIMULATED,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0,0,WELL,WELL-00002
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102,2,0.0,0.0,0,0,0,2,WELL,WELL-00011
